# Vietnam THOR dataset plots

## Links

* https://programminghistorian.org/en/lessons/visualizing-with-bokeh

In [ ]:
import os, io, random
import string
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
from collections import OrderedDict
import datetime as dt
import geopandas as gpd

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS, DatePicker
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.models import (DataTable, GeoJSONDataSource, ColumnDataSource, HoverTool, renderers,
                          Label, LabelSet, CustomJS, MultiSelect, Dropdown, Div)
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
output_notebook()
import panel as pn
import panel.widgets as pnw
pn.extension()

In [ ]:
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):
    """convert mat long to web mercartor"""

    k = 6378137
    df.loc[:,"x"] = df[lon] * (k * np.pi/180.0)
    df.loc[:,"y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [ ]:
#df=pd.read_csv('thor_data_vietnam.csv', low_memory=False)

## get subset of columns

In [ ]:
cols = ['MSNDATE','TGTCOUNTRY','TAKEOFFLOCATION','WEAPONTYPE','MFUNC_DESC','TGTLATDD_DDD_WGS84','TGTLONDDD_DDD_WGS84']
s=df[cols]
s=s.sort_values('MSNDATE')
s.to_csv('thor_data_vietnam_small.csv')

In [ ]:
s=pd.read_csv('thor_data_vietnam_small.csv', low_memory=False,index_col=0)
s['MSNDATE'] = pd.to_datetime(s.MSNDATE, format='%Y/%m/%d',errors='coerce')
s['YEAR'] = s.MSNDATE.dt.year.fillna(0).astype(int)
s=s[s.YEAR>0]
s = wgs84_to_web_mercator(s, lon="TGTLONDDD_DDD_WGS84", lat="TGTLATDD_DDD_WGS84")

In [ ]:
print (s.loc[50])

In [ ]:
x = s[~s.TGTLATDD_DDD_WGS84.isnull()].copy()
x = x[~x.TGTCOUNTRY.isin(['PHILLIPINES','UNKNOWN','WESTPAC WATERS'])]

In [ ]:
c = x.TGTCOUNTRY.value_counts()
c.plot(kind='pie',figsize=(6,6))
print(c)
plt.title('Missions flown per country')
plt.savefig('thor_seasia_summary_totals.jpg',dpi=100)

In [ ]:
w = x.WEAPONTYPE.value_counts()
w[:20].plot(kind='barh',figsize=(8,5))

In [ ]:
x.MFUNC_DESC.value_counts()[:12]

In [ ]:
y=pd.pivot_table(x,index='YEAR',columns=['TGTCOUNTRY'],values='x',aggfunc='size')
y.plot(kind='bar',width=.9,figsize=(12,5))
plt.title('Total missions per year')
plt.savefig('thor_seasia_summary_byyear.jpg',dpi=100)

In [ ]:
#sns.catplot(x,x='year',hue='TGTCOUNTRY')
x[12:14]

## laos

In [ ]:
borders= gpd.read_file('SE_ASIA_PROVINCES_SV_NV_KH_LA.shp')

In [ ]:
borders.plot(column='COUNTRY',cmap='Set3',lw=.5,ec='black',figsize=(10,10))
plt.savefig('seasia_map.jpg',dpi=150)

In [ ]:
laos=x[x.TGTCOUNTRY=='LAOS']
gdf=gpd.GeoDataFrame(laos, geometry=gpd.points_from_xy(laos.TGTLONDDD_DDD_WGS84, laos.TGTLATDD_DDD_WGS84),crs="EPSG:4326")

In [ ]:
laos.MFUNC_DESC.value_counts()[:10]

In [ ]:
lts=laos.groupby('MSNDATE').agg({'MFUNC_DESC':'size'})
lts.plot(figsize=(15,4))


In [ ]:
f,ax=plt.subplots(3,3,figsize=(18,12))
axs=list(ax.flat)
i=0
for y,g in gdf.groupby('YEAR'):
    ax=axs[i]
    borders.plot(column='COUNTRY',cmap='Set2',lw=.2,ec='gray',ax=ax)
    g.plot(color='red',markersize=1,alpha=.5,ax=ax)
    ax.set_xlim(100,110)
    ax.set_ylim(13,23)
    ax.set_title(y)    
    i+=1
f.suptitle('Bombing of Laos by year',fontsize=20)
plt.tight_layout()
plt.savefig('thor_laos_map_byyear.jpg',dpi=150)

## animate the plot

In [154]:
f = plt.figure(figsize=(9,8))
mpl_pane.object = f
for date,g in gdf[:1500].groupby('MSNDATE'):  
    plt.clf()
    f.suptitle('LAOS BOMBING (THOR data)', fontsize=20)
    ax=f.add_subplot(label=date)
    borders.plot(column='COUNTRY',cmap='Set2',lw=.6,ec='gray',ax=ax)
    g.plot(color='red',markersize=5,alpha=.5,ax=ax)
    ax.set_xlim(100,110)
    ax.set_ylim(13,23)
    ax.set_title(date.strftime("%b %d %Y"), fontsize=20)
    mpl_pane.object=f
    mpl_pane.param.trigger('object')
    #ax.clear()
plt.clf();

<Figure size 1296x1152 with 0 Axes>

In [150]:
mpl_pane = pn.pane.Matplotlib(height=600)
mpl_pane

Matplotlib(None, height=600)

## bokeh plots

In [ ]:
colormap={'NORTH VIETNAM':'brown','SOUTH VIETNAM':'orange','LAOS':'red',
                'CAMBODIA':'green','THAILAND':'blue','UNKNOWN':'gray'}
x.MFUNC_DESC.unique()
import matplotlib
cmap = matplotlib.cm.get_cmap('Set1')
#names = cols
#olors = cmap(np.linspace(0, 1, len(names)))
#m={i:c for i, c in zip(names, colors)}

providers = ['CARTODBPOSITRON','STAMEN_TERRAIN','OSM','ESRI_IMAGERY']
cats = ['TGTCOUNTRY','WEAPONTYPE','MFUNC_DESC']

In [ ]:
def draw_map(df=None, long=None, lat=None, height=500, colorby='TGTCOUNTRY',
             point_size=5,
              tile_provider='CARTODBPOSITRON'):
    tile_provider = get_provider(tile_provider)
    tools = "pan,wheel_zoom,box_zoom,hover,tap,lasso_select,reset,save"
    sizing_mode='stretch_both'

    # range bounds supplied in web mercator coordinates
    k = 6378137
    pad = 600000
    if lat == None:
        lat = 14
    if long == None:
        long = 108
    x = long * (k * np.pi/180.0)
    y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
      
    p = figure(x_range=(x-pad, x+pad), y_range=(y-pad, y+pad),
               x_axis_type="mercator", y_axis_type="mercator", tools=tools,
               plot_width=height, plot_height=height, sizing_mode=sizing_mode)
    p.add_tile(tile_provider)
    if df is None:
        return
    df.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in df[colorby]]
    #df['size'] = 10
    source = ColumnDataSource(df)    
    p.circle(x='x', y='y', size=point_size, alpha=0.7, color='color', source=source)#, legend_group=colorby)
    p.toolbar.logo = None    
    p.title.text = "date"
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("TGTCOUNTRY", "@TGTCOUNTRY"),
        ("MSNDATE", "@MSNDATE{%F}"),
        ("TAKEOFFLOCATION", "@TAKEOFFLOCATION"),
        ("WEAPONTYPE", "@WEAPONTYPE"),
        ("MFUNC_DESC", "@MFUNC_DESC")     
    ])
    hover.formatters={'@MSNDATE': 'datetime'}
    return p 

In [ ]:
p = draw_map(x[:5000])
#pn.pane.Bokeh(p)

## dashboard

In [ ]:
cols = list(x.columns)
colorby='TGTCOUNTRY'
map_pane=pn.pane.Bokeh(width=700)
df_pane = pn.pane.DataFrame(width=600,height=600)
tile_select = pnw.Select(name='tile layer',options=providers,width=200)
filterby_select = pnw.Select(name='filter by',value='',options=['']+cols[1:4],width=200)
value_select = pnw.Select(name='value',value='',options=[],width=200)
find_btn = pnw.Button(name='find in region',button_type='primary',width=200)

def update_tile(event=None):
    p = map_pane.object
    p.renderers = [x for x in p.renderers if not str(x).startswith('TileRenderer')]
    rend = renderers.TileRenderer(tile_source= get_provider(tile_select.value))
    p.renderers.insert(0, rend)

def update_filter(event):
    col=filterby_select.value
    if col=='':
        value_select.options = []
    else:
        value_select.options = list(x[col].unique())
    
def find_in_region(event):
    #get points in selected map area
    p = map_pane.object
    d = x[(x.x>p.x_range.start) & (x.x<p.x_range.end) & (x.y>p.y_range.start) & (x.y<p.y_range.end)]
    #print (len(sel))
    if len(d)==0:
        return
    elif len(d)>20000:
        p.title.text = 'too many points!'
        return
    d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]
    source = p.renderers[1].data_source
    source.data = dict(d)
    p.title.text = 'selected %s points' %len(d)
    update_tile()
    #df_pane.object=d
    return

def update_map(event=None):
    
    date = str(date_picker.value)
    #print (val)
    d = x[x.MSNDATE==date]
    col = filterby_select.value
    val = value_select.value
    if col != '':
        d = d[d[col]==val]    
    if len(d)==0:
        return        
  
    d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]  
    p = map_pane.object
    source = p.renderers[1].data_source
    source.data = dict(d)
    p.title.text = date
    
date='1968-01-01'
d = x[x.MSNDATE==date]
map_pane.object=draw_map(d)
#date_picker = pnw.DatePicker(name='Date Picker', value=dt.datetime(1965, 1, 1))
date_picker = pnw.DateSlider(name='Date', start=dt.datetime(1965, 1, 1), 
                             end=dt.datetime(1973, 10, 31), value=dt.datetime(1968, 1, 1))
date_picker.param.watch(update_map,'value')
tile_select.param.watch(update_tile,'value')
filterby_select.param.watch(update_filter,'value')
value_select.param.watch(update_map,'value')
find_btn.on_click(find_in_region)

pn.Column(date_picker,pn.Row(pn.Column(tile_select,filterby_select,value_select,find_btn),map_pane))

## animate

In [ ]:
pane=pn.pane.Bokeh(draw_map(x[:1]),width=400,height=400)
pane

In [ ]:
for i,d in x[:5000].groupby('MSNDATE'):
    print (i)
    p=pane.object
    source = p.renderers[1].data_source  
    source.data=dict(d)
    #p=draw_map(d)